In [ ]:
import os, re
import google.generativeai as genai

os.environ["HF_TOKEN"] = ""
genai.configure(api_key="")

In [2]:
import json
from huggingface_hub import hf_hub_download

tdc_prompts_filepath = hf_hub_download(
    repo_id="google/txgemma-27b-predict",
    filename="tdc_prompts.json",
)

with open(tdc_prompts_filepath, "r") as f:
    tdc_prompts_json = json.load(f)

In [3]:
dataset = "ClinTox"  #@param ["AMES", "ClinTox"]

# now use it:
prompt = tdc_prompts_json[dataset]
print(prompt)

Instructions: Answer the following question about drug properties.
Context: Humans are exposed to a variety of chemicals through food, household products, and medicines, some of which can be toxic, leading to over 30% of promising pharmaceuticals failing in human trials due to toxicity. Toxic drugs can be identified from clinical trials that failed due to toxicity, while non-toxic drugs can be identified from FDA approval status or from clinical trials that report no toxicity.
Question: Given a drug SMILES string, predict whether it
(A) is not toxic (B) is toxic
Drug SMILES: {Drug SMILES}
Answer:


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

PREDICT_VARIANT = "9b-predict"  # @param ["2b-predict", "9b-predict", "27b-predict"]
CHAT_VARIANT = "9b-chat" # @param ["9b-chat", "27b-chat"]
USE_CHAT = True # @param {type: "boolean"}

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

predict_tokenizer = AutoTokenizer.from_pretrained(f"google/txgemma-{PREDICT_VARIANT}")
predict_model = AutoModelForCausalLM.from_pretrained(
    f"google/txgemma-{PREDICT_VARIANT}",
    device_map="auto",
    quantization_config=quantization_config,
)

if USE_CHAT:
    chat_tokenizer = AutoTokenizer.from_pretrained(f"google/txgemma-{CHAT_VARIANT}")
    chat_model = AutoModelForCausalLM.from_pretrained(
        f"google/txgemma-{CHAT_VARIANT}",
        device_map="auto",
        quantization_config=quantization_config,
    )

Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.90s/it]


In [5]:
## Example task and input
task_name = "AMES"
smiles = "{Drug SMILES}"
sequence = "{Target amino acid sequence}"
drug_smiles = "C[C@@H]1Cc2c([nH]c3ccccc23)[C@H](N1CC(F)(F)F)c1c(F)ccc(NCCNCCCF)c1F"
AA_sequence = "MTAEKEKKRCSSERRKEKSRDAARCRRSKETEVFYELAHQLPIPHSISSHLDKASIMRLAISFLRTRKLLTSGCVAATETTDVDRLMDSWYLKPLGGFITVVTSDGDMIFLSENINKFMGLTQVELTGHSIFDFTHPCDHEEIRENLSLKAGMGKKGKELNTERDFFMRMKCTVTNRGRTVNLKSASWKVLHCTGHLKVCNGCPARVLCGFKEPPLTCVVMMCEPIPHPSNIDTPLDSKAFLSRHSMDMKFTYCDDRVTELMGYSPEDLLGRSAYDFYHALDSDNVTKSHQNLCTKGQAVSGQYRMLAKNGGYVWVETQGTVIYNSRNSQPQCIVCVNYVLSDVEEKSMIFSMDQTESLFKPHNLNSFFSPSKRSLGSDQSEALFTKLKEEPEDLTQLAPTPGDTIISLDFGQPQYEEHPMYSKVSSVAPPVSHSIHDGHKASYAGDMPKMAATFSVPQAPPPSSATPSLSSCSTPSSPGDYYTPVDSDLKVELTEKLFSLDTQETKASCNQENDLSDLDLETLAPYIPMDGEDFQLNPICQEEPASEIGGLVTNQQSFSNITSLFQPLGSSSAAHFQPNMSSGGDKKSISGGSVGSWPSIPCSRGPMQMPPYHDPASTPLSSMGGRQNLQWPPDPPLPSKAGMMDPLAAKRSCQTMPANRMPLYLQRPVENFVQNYRDMSPARLALTNGFKRSFTQMTMGESPPTKSQQTLWKRLRNESCAVMDRKSLSTSALSDKGMAHNRGMDHQHRKTQYSGNQTGQAAKCYREQCCNYREFSMQPSSKMDGIASRLIGPSFETYSLPELTRYDCEVNVPLQGNLHLLQGSDLLRALDQST"
TDC_PROMPT = tdc_prompts_json[task_name].replace(smiles, drug_smiles).replace(sequence, AA_sequence)

def txgemma_predict(prompt):
    input_ids = predict_tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = predict_model.generate(**input_ids, max_new_tokens=8)
    return predict_tokenizer.decode(outputs[0], skip_special_tokens=True)

def txgemma_chat(prompt):
    input_ids = chat_tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = chat_model.generate(**input_ids, max_new_tokens=32)
    return chat_tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Prediction model response: {txgemma_predict(TDC_PROMPT)}")
if USE_CHAT: print(f"Chat model response: {txgemma_chat(TDC_PROMPT)}")

/home/ubuntu/.local/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Prediction model response: Instructions: Answer the following question about drug properties.
Context: Mutagenicity means the ability of a drug to induce genetic alterations. Drugs that can cause damage to the DNA can result in cell death or other severe adverse effects. Nowadays, the most widely used assay for testing the mutagenicity of compounds is the Ames experiment which was invented by a professor named Ames. The Ames test is a short-term bacterial reverse mutation assay detecting a large number of compounds which can induce genetic damage and frameshift mutations.
Question: Given a drug SMILES string, predict whether it
(A) is not mutagenic (B) is mutagenic
Drug SMILES: C[C@@H]1Cc2c([nH]c3ccccc23)[C@H](N1CC(F)(F)F)c1c(F)ccc(NCCNCCCF)c1F
Answer:B)
Chat model response: Instructions: Answer the following question about drug properties.
Context: Mutagenicity means the ability of a drug to induce genetic alterations. Drugs that can cause damage to the DNA can result in cell death or

# Tool to allow our Agentic-Tx to ask TxGemma therapeutically relevant questions

Making a tool for our agent to use: a chat interface for our llama-based Agentic-Tx and TxGemma-Chat.

In [6]:
# This will allow us to extract content from inside of ticks
def extract_prompt(text, word):
    code_block_pattern = rf"```{word}(.*?)```"
    code_blocks = re.findall(code_block_pattern, text, re.DOTALL)
    extracted_code = "\n".join(code_blocks).strip()
    return extracted_code

# This class will allow us to inferface with TxGemma
class TxGemmaChatTool:
    def __init__(self):
      self.tool_name = "Chat Tool"

    def use_tool(self, question):
        # Here, we are submitting a question to TxGemma
        response = txgemma_chat(question)
        return response

    def tool_is_used(self, query):
        # This just checks to see if the tool call was evoked
        return "```TxGemmaChat" in query

    def process_query(self, query):
        # Here, we clean to query to remove the tool call
        return extract_prompt(query, word="TxGemmaChat")

    def instructions(self):
        # Here, we are **very** descriptively explaining how the tool works to the agent
        # This will be useful later on
        return (
            "=== Therapeutic Chat Tool Instructions ===\n"
            "### What This Tool Does\n"
            "The Therapeutic Chat Tool allows you to chat with a knowledgeable large language model named TxGemma trained on many therapeutics datasets."
            "### When and Why You Should Use It\n"
            "- If you have therapeutics related questions that you would benefit from asking TxGemma from.\n"
            "### How to Use It\n"
            "Format your query with triple backticks (```), and start with `TxGemmaChat`. Then on a new line:\n"
            "1) **Any question you would like to ask**\n\n"
            "Example:\n"
            "```TxGemmaChat\n"
            "What is a common drug used to treat ovarian cancer?\n"
            "```\n")


# Making a TxGemma prediction

AMES Mutagenicity

In [37]:
# This class will allow us to predict binding affinity using TxGemma
class Mutagenicity:
    def __init__(self):
      self.tool_name = "AMES Mutagenicity Prediction"

    def use_tool(self, smiles_string):
        # Here, we are submitting the smiles to TxGemma, and returning the response
        prediction = txgemma_predict(tdc_prompts_json["AMES"].replace("{Drug SMILES}", smiles_string))
        if "(A)" in prediction:   prediction = f"{smiles_string} is not mutagenic!"
        elif "(B)" in prediction: prediction = f"{smiles_string} is mutagenic!"
        return prediction

    def tool_is_used(self, query):
        # This just checks to see if the tool call was evoked
        return "```MutagenicityPred" in query

    def process_query(self, query):
        # Here, we clean to query to remove the tool call
        return extract_prompt(query, word="MutagenicityPred")

    def instructions(self):
        # Here, we are explaining how the tool works to the agent
        return (
        "=== AMES Mutagenicity Prediction Instructions ===\n"
            "The AMES Mutagenicity Prediction Tool is designed to predict potential for mutagenicity for humans in clinicial trials.\n"
            "You can test the mutagenicity of different SMILES strings as they might affect humans.\n"
            "To properly use this tool, follow the format outlined below:\n"
            "1. **Form a AMES Mutagenicity Prediction query**:\n"
            "```MutagenicityPred\n\n```\n"
            "Example: ```MutagenicityPred\nCN(C)C(=N)N=C(N)N\n```\n"
            "- Replace `` with an exact smiles string. "
            "A result will be returned to you describing the AMES Mutagenicity Prediction.\n"
            "**Important Formatting Details**:\n"
            "- Use `MutagenicityPred` as the exact keyword to begin your query.\n"
            "- Place your text after `MutagenicityPred` on a new line.\n"
            "- Enclose the entire query using three backticks (```), as shown in the example above.\n")

ClinTox

In [32]:
class ClinTox:
    def __init__(self):
      self.tool_name = "Clinical Toxicology Prediction"

    def use_tool(self, smiles_string):
        # Here, we are submitting the smiles to TxGemma, and returning the response
        prediction = txgemma_predict(tdc_prompts_json["AMES"].replace("{Drug SMILES}", smiles_string))
        if "(A)" in prediction:   prediction = f"{smiles_string} is predicted to be toxic!"
        elif "(B)" in prediction: prediction = f"{smiles_string} is predicted to be not toxic!"
        return prediction

    def tool_is_used(self, query):
        # This just checks to see if the tool call was evoked
        return "```ClinToxPred" in query

    def process_query(self, query):
        # Here, we clean to query to remove the tool call
        return extract_prompt(query, word="ClinToxPred")

    def instructions(self):
        # Here, we are explaining how the tool works to the agent
        return (
        "=== Clinical Toxicology Prediction Instructions ===\n"
            "The Clinical Toxicology Prediction Tool is designed to predict potential for toxicology for humans in clinicial trials.\n"
            "You can test the toxicology of different SMILES strings as they might affect humans.\n"
            "To properly use this tool, follow the format outlined below:\n"
            "1. **Form a Clinical Toxicology Prediction query**:\n"
            "```ClinToxPred\n\n```\n"
            "Example: ```ClinToxPred\nCN(C)C(=N)N=C(N)N\n```\n"
            "- Replace `` with an exact smiles string. "
            "A result will be returned to you describing the Clinical Toxicology Prediction.\n"
            "**Important Formatting Details**:\n"
            "- Use `ClinToxPred` as the exact keyword to begin your query.\n"
            "- Place your text after `ClinToxPred` on a new line.\n"
            "- Enclose the entire query using three backticks (```), as shown in the example above.\n")

# PubMed search tool

In [9]:
! pip install --upgrade --quiet biopython

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
from Bio import Medline, Entrez

# This class will allow us to interface with PubMed
class PubMedSearch:
    def __init__(self):
      self.tool_name = "PubMed Search"

    def tool_is_used(self, query: str):
        # This just checks to see if the tool call was evoked
        return "```PubMedSearch" in query

    def process_query(self, query: str):
        # Here, we clean to query to remove the tool call
        search_text = extract_prompt(query, word="PubMedSearch")
        return search_text.strip()

    def use_tool(self, search_text):
        # Here, we are searching through PubMed and returning relevant articles
        pmids = list()
        handle = Entrez.esearch(db="pubmed", sort="relevance", term=search_text, retmax=3)
        record = Entrez.read(handle)
        pmids = record.get("IdList", [])
        handle.close()

        if not pmids:
            return f"No PubMed articles found for '{search_text}' Please try a simpler search query."

        fetch_handle = Entrez.efetch(db="pubmed", id=",".join(pmids), rettype="medline", retmode="text")
        records = list(Medline.parse(fetch_handle))
        fetch_handle.close()

        result_str = f"=== PubMed Search Results for: '{search_text}' ===\n"
        for i, record in enumerate(records, start=1):
            pmid = record.get("PMID", "N/A")
            title = record.get("TI", "No title available")
            abstract = record.get("AB", "No abstract available")
            journal = record.get("JT", "No journal info")
            pub_date = record.get("DP", "No date info")
            authors = record.get("AU", [])
            authors_str = ", ".join(authors[:3])
            result_str += (
                f"\n--- Article #{i} ---\n"
                f"PMID: {pmid}\n"
                f"Title: {title}\n"
                f"Authors: {authors_str}\n"
                f"Journal: {journal}\n"
                f"Publication Date: {pub_date}\n"
                f"Abstract: {abstract}\n")
        return f"Query: {search_text}\nResults: {result_str}"

    def instructions(self):
        # Here, we are explaining how the tool works to the agent
        return (
            f"{'@' * 10}\n@@@ PubMed Search Tool Instructions @@@\n\n"
            "### What This Tool Does\n"
            "The PubMed Search Tool queries the NCBI Entrez API (PubMed) for a given search phrase, "
            "and retrieves metadata for a few of the top articles (PMID, title, authors, journal, date, abstract).\n\n"
            "### When / Why You Should Use It\n"
            "- To find **scientific literature** references on a specific biomedical topic.\n"
            "- To retrieve **abstracts, titles, authors**, and other metadata.\n\n"
            "### Query Format\n"
            "Wrap your request with triple backticks, starting with `PubMedSearch`. For example:\n\n"
            "```PubMedSearch\ncancer immunotherapy\n```\n\n"
            "### Example\n"
            "```PubMedSearch\nmachine learning in drug discovery\n```\n"
            "- This will search PubMed for articles related to 'machine learning in drug discovery', "
            "fetch up to 3 PMIDs, and return their titles, abstracts, etc.\n\n")

In [35]:
pubmed_tool = PubMedSearch()
search_results = pubmed_tool.use_tool("Is aspirin toxic?")
print(search_results)

Query: Is aspirin toxic?
Results: === PubMed Search Results for: 'Is aspirin toxic?' ===

--- Article #1 ---
PMID: 24361050
Title: An aspirin a day.
Authors: Majerus PW
Journal: Advances in biological regulation
Publication Date: 2014 Jan
Abstract: The title of this article is also its punch line. The thesis that I will prove is that every adult, with a few exceptions, should take one 325 mg aspirin tablet each day. The drug is extraordinary and is beneficial in myriad ways. In this dosage the toxicity of the treatment is minimal. Since the drug is sold "over the counter", not requiring prescription, it is cheap and its benefits are easily underestimated. I do not use extensive reference citations; but just tell the story of aspirin.

--- Article #2 ---
PMID: 39092921
Title: Pharmacokinetics of aspirin: evaluating shortcomings in the literature.
Authors: Visagie JL, Aruwajoye GS, van der Sluis R
Journal: Expert opinion on drug metabolism & toxicology
Publication Date: 2024 Aug
Abstract

# Wrapping it all together

### Creating a tool manager

In [11]:
# The tool manager will hold all of the tools, and provide an interface for the agent
class ToolManager:
    def __init__(self, toolset):
        self.toolset = toolset

    def tool_prompt(self):
        # This will let the agent know what tools it has access to
        tool_names = ", ".join([tool.tool_name for tool in self.toolset])
        return f"You have access to the following tools: {tool_names}\n{self.tool_instructions()}. You can only use one tool at a time. These are the only tools you have access to nothing else."

    def tool_instructions(self):
        # This allows the agent to know how to use the tools
        tool_instr = "\n".join([tool.instructions() for tool in self.toolset])
        return f"The following is a set of instructions on how to use each tool.\n{tool_instr}"

    def use_tool(self, query):
        # This will iterate through all of the tools
        # and find the correct tool that the agent requested
        for tool in self.toolset:
            if tool.tool_is_used(query):
                # use the tool and return the output
                return tool.use_tool(tool.process_query(query))
        return f"No tool match for search: {query}"

if USE_CHAT:
    tools = ToolManager([TxGemmaChatTool(), Mutagenicity(), ClinTox(), PubMedSearch()])
else:
    tools = ToolManager([Mutagenicity(), ClinTox(), PubMedSearch()])

### Creating a gemini inference tool

In [28]:
def inference_gemini(prompt, system_prompt, model_str):
  # Check to see that our model string matches
  if model_str == "gemini-2.5-flash":
    model = genai.GenerativeModel(model_name="gemini-2.5-flash-preview-05-20", system_instruction=system_prompt)
    response = model.generate_content(prompt)
    answer = response.text
  return answer

In [29]:
# def inference_llama(prompt: str, system_prompt: str, model_str: str) -> str:
#     if model_str == "llama-3.1-8B":
#         full_prompt = system_prompt.strip() + "\n" + prompt.strip()
#         outputs = pipe(full_prompt, return_full_text=False)
#         return outputs[0]["generated_text"]
#     else:
#         raise ValueError(f"Unsupported model_str: {model_str}")

# Creating a therapeutics agent

In [38]:
# This class defines our Agentic-Tx, wrapping together all of our tools and the orchestrator
class AgenticTx:
  def __init__(self, tool_manager, model_str, num_steps=5):
    self.curr_steps = 0
    self.num_steps = num_steps
    self.model_str = model_str
    self.tool_manager = tool_manager
    self.thoughts = list()
    self.actions  = list()
    self.observations = list()

  def reset(self):
    # Reset the number of steps taken
    self.curr_steps = 0

  def system_prompt(self, use_tools=True):
    # These are the system instructions for AgenticTx
    role_prompt = "You are an expert therapeutic agent. You answer accurately and thoroughly."
    prev_actions = f"You can perform a maximum of {self.num_steps} actions. You have performed {self.curr_steps} and have {self.num_steps - self.curr_steps - 1} left."
    if use_tools: tool_prompt = "You can use tools to solve problems and answer questions. " + self.tool_manager.tool_prompt()
    else: tool_prompt = "You cannot use any tools right now."
    return f"{role_prompt} {prev_actions} {tool_prompt}"

  def prior_information(self, query):
      info_txt = f"Question: {query}\n" if query is not None else ""
      for _i in range(self.curr_steps):
          info_txt += f"### Thought {_i + 1}: {self.thoughts[_i]}\n"
          info_txt += f"### Action {_i + 1}: {self.actions[_i]}\n"
          info_txt += f"### Observation {_i + 1}: {self.observations[_i]}\n\n"
          info_txt += "@"*20
      return info_txt

  def step(self, question):
    for _i in range(self.num_steps):
      if self.curr_steps == self.num_steps-1:
        return inference_gemini(
            model_str=self.model_str,
            prompt=f"{self.prior_information(question)}\nYou must now provide an answer to this question {question}",
            system_prompt=self.system_prompt(use_tools=False))
      else:
        # Provide a thought step, planning for the model
        thought = inference_gemini(
            model_str=self.model_str,
            prompt=f"{self.prior_information(question)}\nYou cannot currently use tools but you can think about the problem and what tools you want to use. This was the question, think about plans for how to use tools to answer this {question}. Let's think step by step (respond with only 1-2 sentences).\nThought: ",
            system_prompt=self.system_prompt(use_tools=False))
        # Provide a took action for the model
        action = inference_gemini(
            model_str=self.model_str,
            prompt=f"{self.prior_information(question)}\n{thought}\nNow you must use tools to answer the following user query [{question}], closely following the tool instructions. Tool",
            system_prompt=self.system_prompt(use_tools=True))
        obs = self.tool_manager.use_tool(action)

        print("Thought:", thought)
        print("Action:",  action)
        print("Observation:",  obs)

        self.thoughts.append(thought)
        self.actions.append(action)
        self.observations.append(obs)

        self.curr_steps += 1


agentictx = AgenticTx(tool_manager=tools, model_str="gemini-2.5-flash")
# It should select CS(=O)(=O)C1=C2[C@@H]([C@@H]([C@@H](C2=C(C=C1)OC3=CC(=CC(=C3)C#N)F)F)F)O because it is a known HIF-2a binder
response = agentictx.step("Is this drug toxic COc1c(F)cccc1C1=C(c2ccc(O[C@H]3CCN(CCCF)C3)cc2)c2ccc(O)cc2CCC1? If it is toxic, what are the properties that makes it toxic")
print("\nFinal Response:", response)

Thought: My plan would be to first use cheminformatics tools to calculate key physicochemical properties and predict ADMET parameters (absorption, distribution, metabolism, excretion, toxicity), including potential toxicophores or structural alerts. I would then leverage predictive models and databases to assess the compound's overall toxicity profile based on these properties and any similar known compounds.
Action: ```ClinToxPred
COc1c(F)cccc1C1=C(c2ccc(O[C@H]3CCN(CCCF)C3)cc2)c2ccc(O)cc2CCC1
```
Observation: COc1c(F)cccc1C1=C(c2ccc(O[C@H]3CCN(CCCF)C3)cc2)c2ccc(O)cc2CCC1 is not mutagenic!
Thought: To determine if a drug is toxic and identify the properties contributing to toxicity, one would typically use a multi-pronged approach involving computational predictions and structural analysis.

**Plan for Toxicity Assessment:**

1.  **Structural Analysis for Alerts:** Examine the SMILES string for common toxicophores or structural alerts known to be associated with various toxicities (e.g